## Homework 5
Cody Costa\
9/5/2025

<hr>

# Problem 2

updating CNN weights

In [2]:
import numpy as np

# define sigmoid and its derivative function

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(y):

    # y = sigmoid(x)
    return y * (1 - y)


# starting data

x = np.array([0.25, 0.275])     # given inputs
t = np.array([0.875, 0.125])    # target outputs


# initial weights

# 1. Input --> Hidden

W_IH = np.array([
    [0.25, 0.375, 0.5],     # h1
    [0.125, 0.625, 0.125]   # h2
])

# 2. Hidden --> Output

W_HO = np.array([
    [0.5, 0.75, 0.25],      # o1
    [0.375, 0.875, 0.5]     # o2
])


# Forward Pass

# add the bias inputs
x_bias = np.append(x, 1)

# hidden layer
net_h = W_IH @ x_bias
h = sigmoid(net_h)
h_bias = np.append(h, 1)

# output layer
net_o = W_HO @ h_bias
o = sigmoid(net_o)

print(f'Forward Pass Outputs: {o}\n')


# Backward Pass

# output delta
delta_o = (t - o) * sigmoid_derivative(o)

# hidden delta
delta_h = (W_HO[:, :-1].T @ delta_o) * sigmoid_derivative(h)


# Update Weights

# Hidden --> Output
dW_HO = np.outer(delta_o, h_bias)

# Input --> Hidden
dW_IH = np.outer(delta_h, x_bias)

# apply update
W_HO_new = W_HO + dW_HO
W_IH_new = W_IH + dW_IH


print(f'New input --> hidden weights: \n{W_IH_new}\n')
print(f'New hidden --> output weights: \n{W_HO_new}')

Forward Pass Outputs: [0.7342393  0.77839477]

New input --> hidden weights: 
[[0.24840054 0.3732406  0.49360217]
 [0.12025271 0.61977798 0.10601085]]

New hidden --> output weights: 
[[0.51814252 0.7659666  0.27746691]
 [0.30055365 0.80948237 0.38729179]]


<hr>

# Problem 3

Animals-2 simplification

### Step 1: Filter Dataset to Only Cats and Chickens

In [ ]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# animals data directory
dataset_dir = f'{os.path.expanduser('~')}\\Downloads\\archive\\raw-img'
# print(dataset_dir)

selected_classes = {
    'cat': 0,
    'chicken': 1
}

# function to parse, load, and resize img files in dataset folder
def load_img(directory, classes):
    Images = []
    Labels = []

    for label_name, label_code in classes.items():
        folder = os.path.join(directory, label_name)

        for image_file in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder, image_file))
                img = cv2.resize(img, (128, 128))
                Images.append(img)
                Labels.append(label_code)

            except:
                continue

    return shuffle(np.array(Images), np.array(Labels), random_state=42)


# Load cats and chickens images
x, y = load_img(dataset_dir, selected_classes)

print(f'dataset shape:  [{x.shape}, {y.shape}]')
print(f'uniques:    {np.unique(y)}')

dataset shape:  [(4766, 128, 128, 3), (4766,)]
uniques:    [0 1]


### Step 2: Normalize / split data

In [5]:
x = x.astype('float32') / 255

# split data into training, validation, and testing
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)


print(f'Training set:   {x_train.shape}, {y_train.shape}')
print(f'Validation set: {x_val.shape}, {y_val.shape}')
print(f'Testing set:    {x_test.shape}, {y_test.shape}')

Training set:   (3336, 128, 128, 3), (3336,)
Validation set: (715, 128, 128, 3), (715,)
Testing set:    (715, 128, 128, 3), (715,)


### Step 3: Train the CNN with 3 convolutional layers

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = keras.models.Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# added conv layer
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))  
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # binary output

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_val, y_val))

c:\Users\Cody Costa\OneDrive\Desktop\SJSU-MSE\EE263\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 131ms/step - accuracy: 0.7230 - loss: 0.5395 - val_accuracy: 0.8308 - val_loss: 0.3969
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.8348 - loss: 0.3954 - val_accuracy: 0.8280 - val_loss: 0.4028
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.8561 - loss: 0.3509 - val_accuracy: 0.8476 - val_loss: 0.3707
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 127ms/step - accuracy: 0.8777 - loss: 0.2953 - val_accuracy: 0.8881 - val_loss: 0.2907
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 125ms/step - accuracy: 0.8966 - loss: 0.2589 - val_accuracy: 0.8629 - val_loss: 0.3175
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.9125 - loss: 0.2151 - val_accuracy: 0.8951 - val_loss: 0.2787
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.9293 - loss: 0.1851 - val_accuracy: 0.8979 - val_loss: 0.2607
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 13s 128ms/step - accuracy: 0.9412 - loss: 0

### Step 4: Report Accuracy

In [7]:
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Accuracy: {acc:.3f}")


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8979 - loss: 0.2736
Test Accuracy: 0.898
